In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Concatenate
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input
from keras import backend as K
from keras.models import load_model
import pickle
import pandas as pd
import re
import numpy as np
import random
from keras.utils.vis_utils import plot_model
import keras.callbacks
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn import datasets, linear_model
import seaborn as sns
from numpy.random import seed; seed(123)
from tensorflow import set_random_seed; set_random_seed(123)
from sklearn.metrics import roc_auc_score
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import kruskal
from scipy.stats import ttest_ind
from sklearn.tree import DecisionTreeRegressor

Using TensorFlow backend.


In [2]:
##### load data #####
processed_data = pickle.load( open( "result_04_processed_data_no_scale_Final.obj", "rb" ) )
cytof_files = processed_data["cytof_files"]
cytof_files.to_csv("result_05_cytof_files.csv")
expr_list = processed_data["expr_list"]

r1 = [t1==t1 for t1 in cytof_files.CMV_Ab] 
cytof_files = cytof_files.loc[r1,:]
expr_list = expr_list[r1]
print(expr_list.shape)

y = cytof_files.CMV_Ab.values> 2
x = expr_list

(532, 10000, 27, 1)


In [3]:
##### split train, validation and test######

train_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i] not in ["SDY515","SDY519"]]
valid_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]=="SDY515"]
test_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]=="SDY519"]

x_train = x[train_id]
x_valid = x[valid_id]
x_test = x[test_id]

y_train = y[train_id]
y_valid = y[valid_id]
y_test = y[test_id]

In [4]:
##### define model #####
model_input = Input(shape=x_train[0].shape)
model_output = Conv2D(3, kernel_size=(1, x_train.shape[2]),
                 activation=None)(model_input)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)
#model_output = Dropout(0.3)(model_output)

model_output = Conv2D(3, (1, 1), activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

model_output = AveragePooling2D(pool_size=(10000, 1))(model_output)
model_output = Flatten()(model_output)

model_output = Dense(3, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)
model_output = Dense(1, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("sigmoid")(model_output)

checkpointer = keras.callbacks.ModelCheckpoint(filepath='result_11_weights.hdf5', monitor='val_loss', 
                                               verbose=0, save_best_only=True)
earlyStop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.00000001, patience=100, 
                                          verbose=0, mode='auto', baseline=None, restore_best_weights=True)

In [ ]:
##### 10 fold test######
valid_sdy = cytof_files.study_accession.unique()
result = pd.DataFrame(columns=['valid', 'test', 'auc'])

for i in range(10):
    sdy = np.random.choice(valid_sdy, size=2, replace=False)
    train_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i] not in sdy]
    valid_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]==sdy[0]]
    test_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]==sdy[1]]

    x_train = x[train_id]
    x_valid = x[valid_id]
    x_test = x[test_id]

    y_train = y[train_id]
    y_valid = y[valid_id]
    y_test = y[test_id]
    
    model = keras.models.Model(inputs=[model_input],
                           outputs=model_output)
    
    model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

    model.fit([x_train], y_train,
          batch_size=60,
          epochs=10000,
          verbose=1,
          callbacks=[checkpointer,earlyStop],
          validation_data=([x_valid], y_valid))

    best_model = load_model('result_11_weights.hdf5')

    y_true = y_test
    y_scores = best_model.predict([x_test])
    result = result.append({'valid':sdy[0] , 'test': sdy[1], 'auc': roc_auc_score(y_true, y_scores)}, 
                           ignore_index=True)
    
result.to_csv("result_11_10fold_test.csv",index=False)

Train on 508 samples, validate on 13 samples
Epoch 1/10000
508/508 [==============================] - 5s 10ms/step - loss: 0.7875 - acc: 0.5020 - val_loss: 0.7541 - val_acc: 0.4615
Epoch 2/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.7812 - acc: 0.5157 - val_loss: 0.7489 - val_acc: 0.4615
Epoch 3/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.7873 - acc: 0.5177 - val_loss: 0.7462 - val_acc: 0.4615
Epoch 4/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.7890 - acc: 0.5177 - val_loss: 0.7438 - val_acc: 0.4615
Epoch 5/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.7862 - acc: 0.5197 - val_loss: 0.7411 - val_acc: 0.4615
Epoch 6/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.7787 - acc: 0.5295 - val_loss: 0.7403 - val_acc: 0.4615
Epoch 7/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.7911 - acc: 0.5236 - val_loss: 0.7397 - val_acc: 0.4615
Epoch 8/1

Epoch 61/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.7180 - acc: 0.5630 - val_loss: 0.7108 - val_acc: 0.5385
Epoch 62/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.7110 - acc: 0.5512 - val_loss: 0.7091 - val_acc: 0.5385
Epoch 63/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.7134 - acc: 0.5610 - val_loss: 0.7073 - val_acc: 0.5385
Epoch 64/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.7150 - acc: 0.5591 - val_loss: 0.7063 - val_acc: 0.5385
Epoch 65/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.7223 - acc: 0.5650 - val_loss: 0.7050 - val_acc: 0.5385
Epoch 66/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.7054 - acc: 0.5571 - val_loss: 0.7032 - val_acc: 0.5385
Epoch 67/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.7018 - acc: 0.5748 - val_loss: 0.7014 - val_acc: 0.5385
Epoch 68/10000
508/508 [========================

508/508 [==============================] - 5s 9ms/step - loss: 0.6697 - acc: 0.5965 - val_loss: 0.6777 - val_acc: 0.5385
Epoch 122/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.6596 - acc: 0.5945 - val_loss: 0.6774 - val_acc: 0.5385
Epoch 123/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.6652 - acc: 0.6122 - val_loss: 0.6781 - val_acc: 0.5385
Epoch 124/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.6605 - acc: 0.6102 - val_loss: 0.6777 - val_acc: 0.5385
Epoch 125/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.6597 - acc: 0.6122 - val_loss: 0.6763 - val_acc: 0.5385
Epoch 126/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.6576 - acc: 0.6122 - val_loss: 0.6762 - val_acc: 0.5385
Epoch 127/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.6581 - acc: 0.6142 - val_loss: 0.6759 - val_acc: 0.5385
Epoch 128/10000
508/508 [==============================] 

Epoch 181/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.6178 - acc: 0.6634 - val_loss: 0.6693 - val_acc: 0.4615
Epoch 182/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.6253 - acc: 0.6555 - val_loss: 0.6704 - val_acc: 0.4615
Epoch 183/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.6211 - acc: 0.6614 - val_loss: 0.6693 - val_acc: 0.4615
Epoch 184/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.6222 - acc: 0.6575 - val_loss: 0.6667 - val_acc: 0.4615
Epoch 185/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.6152 - acc: 0.6673 - val_loss: 0.6671 - val_acc: 0.4615
Epoch 186/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.6143 - acc: 0.6634 - val_loss: 0.6666 - val_acc: 0.4615
Epoch 187/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.6164 - acc: 0.6673 - val_loss: 0.6666 - val_acc: 0.4615
Epoch 188/10000
508/508 [================

Epoch 241/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.5895 - acc: 0.6870 - val_loss: 0.6513 - val_acc: 0.5385
Epoch 242/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.5896 - acc: 0.6909 - val_loss: 0.6518 - val_acc: 0.5385
Epoch 243/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.5890 - acc: 0.6791 - val_loss: 0.6509 - val_acc: 0.5385
Epoch 244/10000
508/508 [==============================] - 4s 9ms/step - loss: 0.5823 - acc: 0.6949 - val_loss: 0.6510 - val_acc: 0.5385
Epoch 245/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.5867 - acc: 0.6969 - val_loss: 0.6502 - val_acc: 0.5385
Epoch 246/10000
508/508 [==============================] - 4s 8ms/step - loss: 0.5861 - acc: 0.6949 - val_loss: 0.6488 - val_acc: 0.5385
Epoch 247/10000
508/508 [==============================] - 5s 9ms/step - loss: 0.5866 - acc: 0.6890 - val_loss: 0.6486 - val_acc: 0.5385
Epoch 248/10000
508/508 [================

...

Epoch 681/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2824 - acc: 0.8949 - val_loss: 0.4590 - val_acc: 0.7975
Epoch 682/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2685 - acc: 0.8859 - val_loss: 0.4613 - val_acc: 0.7848
Epoch 683/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2395 - acc: 0.9159 - val_loss: 0.4734 - val_acc: 0.7595
Epoch 684/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2610 - acc: 0.8919 - val_loss: 0.4812 - val_acc: 0.7595
Epoch 685/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2737 - acc: 0.8769 - val_loss: 0.4752 - val_acc: 0.7595
Epoch 686/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2665 - acc: 0.8919 - val_loss: 0.4853 - val_acc: 0.7468
Epoch 687/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2568 - acc: 0.8949 - val_loss: 0.4845 - val_acc: 0.7468
Epoch 688/10000
333/333 [================

Epoch 741/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2837 - acc: 0.8739 - val_loss: 0.4275 - val_acc: 0.8228
Epoch 742/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2858 - acc: 0.8769 - val_loss: 0.4200 - val_acc: 0.8228
Epoch 743/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2435 - acc: 0.9009 - val_loss: 0.4158 - val_acc: 0.8228
Epoch 744/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2547 - acc: 0.8979 - val_loss: 0.4188 - val_acc: 0.8228
Epoch 745/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2501 - acc: 0.9069 - val_loss: 0.4210 - val_acc: 0.8228
Epoch 746/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2640 - acc: 0.8979 - val_loss: 0.4303 - val_acc: 0.8228
Epoch 747/10000
333/333 [==============================] - 3s 9ms/step - loss: 0.2532 - acc: 0.9009 - val_loss: 0.4402 - val_acc: 0.8228
Epoch 748/10000
333/333 [================

Epoch 45/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2878 - acc: 0.8853 - val_loss: 0.6960 - val_acc: 0.7250
Epoch 46/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2766 - acc: 0.8878 - val_loss: 0.7106 - val_acc: 0.7167
Epoch 47/10000
401/401 [==============================] - 4s 10ms/step - loss: 0.2833 - acc: 0.8903 - val_loss: 0.7410 - val_acc: 0.7083
Epoch 48/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2855 - acc: 0.8803 - val_loss: 0.7386 - val_acc: 0.7000
Epoch 49/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2940 - acc: 0.8753 - val_loss: 0.7226 - val_acc: 0.7167
Epoch 50/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2710 - acc: 0.8828 - val_loss: 0.7431 - val_acc: 0.7000
Epoch 51/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2942 - acc: 0.8728 - val_loss: 0.7345 - val_acc: 0.7083
Epoch 52/10000
401/401 [=======================

401/401 [==============================] - 4s 9ms/step - loss: 0.2959 - acc: 0.8753 - val_loss: 0.7572 - val_acc: 0.7000
Epoch 106/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2908 - acc: 0.8803 - val_loss: 0.7457 - val_acc: 0.7000
Epoch 107/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2779 - acc: 0.8903 - val_loss: 0.7587 - val_acc: 0.7000
Epoch 108/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.3051 - acc: 0.8728 - val_loss: 0.7539 - val_acc: 0.7000
Epoch 109/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2712 - acc: 0.9077 - val_loss: 0.7547 - val_acc: 0.7000
Epoch 110/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2901 - acc: 0.8828 - val_loss: 0.7265 - val_acc: 0.7083
Epoch 111/10000
401/401 [==============================] - 4s 9ms/step - loss: 0.2786 - acc: 0.8853 - val_loss: 0.6971 - val_acc: 0.7167
Epoch 112/10000
401/401 [==============================] 

394/394 [==============================] - 3s 8ms/step - loss: 0.3591 - acc: 0.8528 - val_loss: 0.3413 - val_acc: 0.8169
Epoch 6/10000
394/394 [==============================] - 3s 9ms/step - loss: 0.3399 - acc: 0.8655 - val_loss: 0.3377 - val_acc: 0.8310
Epoch 7/10000
394/394 [==============================] - 3s 9ms/step - loss: 0.3827 - acc: 0.8401 - val_loss: 0.3335 - val_acc: 0.8310
Epoch 8/10000
394/394 [==============================] - 3s 8ms/step - loss: 0.3431 - acc: 0.8503 - val_loss: 0.3318 - val_acc: 0.8310
Epoch 9/10000
394/394 [==============================] - 3s 8ms/step - loss: 0.3483 - acc: 0.8528 - val_loss: 0.3288 - val_acc: 0.8592
Epoch 10/10000
394/394 [==============================] - 3s 8ms/step - loss: 0.3270 - acc: 0.8706 - val_loss: 0.3346 - val_acc: 0.8310
Epoch 11/10000
394/394 [==============================] - 3s 8ms/step - loss: 0.3325 - acc: 0.8655 - val_loss: 0.3373 - val_acc: 0.8310
Epoch 12/10000
394/394 [==============================] - 3s 8ms/st